In [1]:
%load_ext watermark
%watermark

Last updated: 2022-02-18T16:00:25.615209+01:00

Python implementation: CPython
Python version       : 3.7.6
IPython version      : 7.12.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 23 Model 8 Stepping 2, AuthenticAMD
CPU cores   : 12
Architecture: 64bit



In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

%matplotlib qt

In [2]:
import scipy.fftpack
from scipy.signal import gaussian

In [3]:
%run fct_RF_heating
%matplotlib qt

ERROR:root:File `'fct_RF_heating.py'` not found.


# GiantMol

In [4]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide


r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+
Omega = 2.045e6*2*pi # 2.047e6
bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23          # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [7]:
Udc = 1
Urf = 150/4
omega_z_2 = 2*C_e*(mkappa)*Udc/m_Ca/d0**2
omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
ax = 8*C_e*0/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)

alpha = 0.10954803101922947
qx = 0.3499226136397147
ax = 0.0
az = 0.0033534250473806
f_z = 83738.00625693168
f_x = 252999.8876875538
f_r = 245973.40572503334


In [16]:
# voltages for spherical cloud
q   = 0.6
Urf = m_Ca*r0**2*Omega**2/(4*C_e)*q
Udc = d0**2/r0**4*(C_e/(kappa_simion*m_Ca*Omega**2))*Urf**2
print(Urf,Udc)

64.29993125041733 22.822357244989803


In [24]:
n_dt = 1000
dt = 2*pi / (n_dt*Omega)
eta = 1e-18
eta_mass = eta/m_Ca

w_p   = (exp(-eta_mass*dt)+eta_mass*dt-1.0) /(eta_mass*dt*(1.0-exp(-eta_mass*dt)))
w_m   = 1-w_p
print(w_p,w_m)

0.5006123076255327 0.49938769237446734


In [13]:
omega_z_2_fit = (2*pi*90806.9982303)**2
kappa_fit = m_Ca*d0**2*omega_z_2_fit/(2*C_e*1)
print(f'{kappa_fit:.30f}')

0.270471333604033359332419195198


In [13]:
matplotlib.rcParams['figure.figsize'] = [11.69,8.27]
matplotlib.rcParams.update({'font.size': 25})

In [77]:
q = linspace(0,1.5,500)

a0 = -0.5*q**2 + 7*q**4/128 - 29*q**6/2304 + 68687*q**8/18874368
b1 = 1 - q - q**2/8 + q**3/64 - q**4/1536 - 11*q**5/36864 + 49*q**6/589824 - 55*q**7/9437184 - 83*q**8/3539440

nutest = linspace(0.1,0.9,9)
print(nutest)
a = []
for nu in nutest:
    a.append( nu**2 + q**2/(2*(nu**2-1)) + (5*nu**2+7)*q**4/(32*(nu**2-1)**3*(nu**2-4)) + (9*nu**4+58*nu**2+29)*q**6/(64*(nu**2-1)**5*(nu**2-4)*(nu**2-9)) )

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]


In [78]:
# plot with complicated scan plot only crystal part at the end

fig = figure('stab_diag',clear='True')
ax = fig.add_subplot(1, 1, 1)
fig.set_size_inches(11.69,8.27)


ax.plot(q,a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
ax.plot(q,b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)
# ax.plot(q,-a0,'-',color='k',label=r'$\Phi_z^{\mathrm{quad}}$ Quadratic pot.',lw=2)
# ax.plot(q,-b1,'-',color='k',label='$\Phi_z^{\mathrm{GM}}$ GiantMol axial pot.',lw=2)

for k in range(len(nutest)):
    ax.plot(q,a[k],color='k',ls=':')

ax.xaxis.set_major_locator(matplotlib.ticker.MultipleLocator(base=0.2))
# text(-0.83, 8000, f'FWHM = ${2*sqrt(2*log(2)) * sig[to_plot] * 1000:.2f}$ MHz\n        T = {temperature:.2f} K',
# rotation=0,
# horizontalalignment='left',
# verticalalignment='bottom',
# color='xkcd:vermillion',
# fontsize=17,
#      bbox=dict(facecolor='white'))

ax.set_xlim(-0.02,1.51)
ax.set_ylim(-1.05,1.05)
# ax.set_xlabel(r'$z$ [mm]')
# ax.set_ylabel('$U_{pot}$ [eV]')
# ax.legend(fontsize=16)

ax.grid()
tight_layout()

# savefig('pot_compare_x1',dpi=300)

# TADOTI

In [10]:
r0=3.93e-3 # 2.5e-3
d0=21e-2/2

Omega=5.215e6*2*pi # 2.047e6

mkappa = 9.6*1e-4

In [11]:
Udc = 1200
Urf = 830/4
omega_z_2 = 2*C_e*mkappa*Udc/m_Ca/d0**2
omega_x_2 = 2*(C_e*Urf/m_Ca/Omega/r0**2)**2
omega_r_2 = omega_x_2 - 0.5*omega_z_2

qx = 4*C_e*Urf/(m_Ca*r0**2*Omega**2)
az = 2*omega_z_2/Omega**2

print('alpha =',omega_z_2/omega_x_2)
print('qx =',qx)
print('ax =',ax)
print('az =',az)
print('f_z =',sqrt(omega_z_2)/2/pi)
print('f_x =',sqrt(omega_x_2)/2/pi)
print('f_r =',sqrt(omega_r_2)/2/pi)

alpha = 0.00025820765491457704
qx = 0.12048466060143968
ax = 0.0
az = 9.370713053118921e-07
f_z = 3569.6499870649727
f_x = 222147.3198086697
f_r = 222132.97931117812
